In [1]:
from PIL import Image, ImageEnhance, ImageFilter  
import os
from glob import glob

import pandas as pd
import numpy as np
import random

In [2]:
def create_folder(path, folder = ""):
    
    final_path = os.path.join(path, folder)
    
    if not os.path.isdir(final_path):
        os.makedirs (final_path)
    
    return final_path

In [3]:
def images_count(path_folder):
    lst = os.listdir(path_folder) 
    count = len(lst)
    return count

In [4]:
# https://www.codementor.io/@isaib.cicourel/image-manipulation-in-python-du1089j1u

# Return color value depending on quadrant and saturation
def get_saturation(value, quadrant):
  if value > 223:
    return 255
  elif value > 159:
    if quadrant != 1:
      return 255

    return 0
  elif value > 95:
    if quadrant == 0 or quadrant == 3:
      return 255

    return 0
  elif value > 32:
    if quadrant == 1:
      return 255

    return 0
  else:
    return 0

# Get the pixel from the given image
def get_pixel(image, i, j):
  # Inside image bounds?
  width, height = image.size
  if i > width or j > height:
    return None

  # Get Pixel
  pixel = image.getpixel((i, j))
  return pixel

# Create a dithered version of the image
def convert_dithering(image):
    # Get size
    width, height = image.size

    # Create new Image and a Pixel Map
    new = create_image_white(width, height)
    pixels = new.load()

    # Transform to half tones
    for i in range(0, width, 2):
        for j in range(0, height, 2):
          # Get Pixels
          p1 = get_pixel(image, i, j)
          p2 = get_pixel(image, i, j + 1)
          p3 = get_pixel(image, i + 1, j)
          p4 = get_pixel(image, i + 1, j + 1)

          # Color Saturation by RGB channel
          red   = (p1[0] + p2[0] + p3[0] + p4[0]) / 4
          green = (p1[1] + p2[1] + p3[1] + p4[1]) / 4
          blue  = (p1[2] + p2[2] + p3[2] + p4[2]) / 4

          # Results by channel
          r = [0, 0, 0, 0]
          g = [0, 0, 0, 0]
          b = [0, 0, 0, 0]

          # Get Quadrant Color
          for x in range(0, 4):
            r[x] = get_saturation(red, x)
            g[x] = get_saturation(green, x)
            b[x] = get_saturation(blue, x)

          # Set Dithered Colors
          pixels[i, j]         = (r[0], g[0], b[0])
          pixels[i, j + 1]     = (r[1], g[1], b[1])
          pixels[i + 1, j]     = (r[2], g[2], b[2])
          pixels[i + 1, j + 1] = (r[3], g[3], b[3])

    # Return new image
    return new

In [5]:
def convert_brightness(image, brightness):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness)
    

In [6]:
def image_name_brightness(name, brightness):
    return ''.join([name, "_", f"{brightness}".replace('.', '_')])

In [7]:
def image_name_dithering(name):
    return ''.join([name, "_dither"])

In [8]:
def image_name_low_quality(name):
    return ''.join([name, "_low"])

In [9]:
def save_image(img, path, name, fillcolor,  quality=75, extension='png'):

    
    img = img.rotate(random.randint(5, 175), Image.NEAREST, expand = False, fillcolor = fillcolor)
    img.save(f"{path}/{name}.{extension}", quality=quality)

In [10]:
# Create a new image with the given size
def create_image_white(i, j):
  image = Image.new("RGB", (i, j), "white")
  return image

In [11]:
def create_image(path, name, r, g, b, override = False, width = 128, height = 128, brightness = 1, extension='png'):
    print(path, name, r, g, b, brightness)
    create_folder(path)
    
    name_brightness = image_name_brightness(name, brightness)
    name_dithering = image_name_dithering(name_brightness)
    name_low_quality = image_name_low_quality(name_brightness)
    
    if glob(f"{path}/{image_name_brightness}.{extension}"):
        #print("already exists")
        if override:
            pass
            
    else:
        #print("let's go !")
        img  = Image.new( mode = "RGB", size = (width, height), color = (r, g, b) )
        
        img = convert_brightness(img, brightness)
        save_image(img, path, name_brightness, (r, g, b))
        
         
        save_image(img, path, name_low_quality, (r, g, b), 1)
        
        # Baisse les perfs
        #img = convert_dithering(img)
        #save_image(img, path, name_dithering, (r, g, b))
        
        
    

In [12]:

df = pd.read_csv("./dataset/html_color_names.csv", sep=";")

df.head()

,color,enable,color_name,hex_code,r,g,b,remark
0,red,1,IndianRed,#CD5C5C,205,92,92,NaN
1,red,1,LightCoral,#F08080,240,128,128,NaN
2,red,1,Salmon,#FA8072,250,128,114,NaN
3,red,0,DarkSalmon,#E9967A,233,150,122,NaN
4,red,0,LightSalmon,#FFA07A,255,160,122,NaN


In [13]:
images_folder = "./dataset/images/"

In [14]:

for index, row in df.iterrows():
    for i in range(9, 14):
        if row['enable'] == 1:
            create_image(
                f"{images_folder}{row['color']}", 
                row['color_name'], 
                r = row['r'], 
                g = row['g'], 
                b = row['b'], 
                brightness=i/10)
    

./dataset/images/red IndianRed 205 92 92 0.9
./dataset/images/red IndianRed 205 92 92 1.0
./dataset/images/red IndianRed 205 92 92 1.1
./dataset/images/red IndianRed 205 92 92 1.2
./dataset/images/red IndianRed 205 92 92 1.3
./dataset/images/red LightCoral 240 128 128 0.9
./dataset/images/red LightCoral 240 128 128 1.0
./dataset/images/red LightCoral 240 128 128 1.1
./dataset/images/red LightCoral 240 128 128 1.2
./dataset/images/red LightCoral 240 128 128 1.3
./dataset/images/red Salmon 250 128 114 0.9
./dataset/images/red Salmon 250 128 114 1.0
./dataset/images/red Salmon 250 128 114 1.1
./dataset/images/red Salmon 250 128 114 1.2
./dataset/images/red Salmon 250 128 114 1.3
./dataset/images/red Crimson 220 20 60 0.9
./dataset/images/red Crimson 220 20 60 1.0
./dataset/images/red Crimson 220 20 60 1.1
./dataset/images/red Crimson 220 20 60 1.2
./dataset/images/red Crimson 220 20 60 1.3
./dataset/images/red Red 255 0 0 0.9
./dataset/images/red Red 255 0 0 1.0
./dataset/images/red Red 2

./dataset/images/purple DarkViolet 148 0 211 1.3
./dataset/images/purple DarkOrchid 153 50 204 0.9
./dataset/images/purple DarkOrchid 153 50 204 1.0
./dataset/images/purple DarkOrchid 153 50 204 1.1
./dataset/images/purple DarkOrchid 153 50 204 1.2
./dataset/images/purple DarkOrchid 153 50 204 1.3
./dataset/images/purple DarkMagenta 139 0 139 0.9
./dataset/images/purple DarkMagenta 139 0 139 1.0
./dataset/images/purple DarkMagenta 139 0 139 1.1
./dataset/images/purple DarkMagenta 139 0 139 1.2
./dataset/images/purple DarkMagenta 139 0 139 1.3
./dataset/images/purple Purple 128 0 128 0.9
./dataset/images/purple Purple 128 0 128 1.0
./dataset/images/purple Purple 128 0 128 1.1
./dataset/images/purple Purple 128 0 128 1.2
./dataset/images/purple Purple 128 0 128 1.3
./dataset/images/purple Indigo 75 0 130 0.9
./dataset/images/purple Indigo 75 0 130 1.0
./dataset/images/purple Indigo 75 0 130 1.1
./dataset/images/purple Indigo 75 0 130 1.2
./dataset/images/purple Indigo 75 0 130 1.3
./data

./dataset/images/blue RoyalBlue 65 105 225 1.1
./dataset/images/blue RoyalBlue 65 105 225 1.2
./dataset/images/blue RoyalBlue 65 105 225 1.3
./dataset/images/blue Blue 0 0 255 0.9
./dataset/images/blue Blue 0 0 255 1.0
./dataset/images/blue Blue 0 0 255 1.1
./dataset/images/blue Blue 0 0 255 1.2
./dataset/images/blue Blue 0 0 255 1.3
./dataset/images/blue MediumBlue 0 0 205 0.9
./dataset/images/blue MediumBlue 0 0 205 1.0
./dataset/images/blue MediumBlue 0 0 205 1.1
./dataset/images/blue MediumBlue 0 0 205 1.2
./dataset/images/blue MediumBlue 0 0 205 1.3
./dataset/images/blue DarkBlue 0 0 139 0.9
./dataset/images/blue DarkBlue 0 0 139 1.0
./dataset/images/blue DarkBlue 0 0 139 1.1
./dataset/images/blue DarkBlue 0 0 139 1.2
./dataset/images/blue DarkBlue 0 0 139 1.3
./dataset/images/blue Navy 0 0 128 0.9
./dataset/images/blue Navy 0 0 128 1.0
./dataset/images/blue Navy 0 0 128 1.1
./dataset/images/blue Navy 0 0 128 1.2
./dataset/images/blue Navy 0 0 128 1.3
./dataset/images/blue Midnigh

In [15]:
#files_count(images_folder+'yellow')

max_images_count = 2000

for c in next(os.walk(images_folder))[1]:
    full_path = images_folder+c #images_folder+c
    img_count = images_count(full_path) # files_color_count = 
    print(c, img_count)
    
    for i in range(img_count, max_images_count):
        
        randint_min = 0
        if i % 2 == 0 or i % 3 == 0:
            randint_min = img_count-1 - i
        
        image_to_copy = os.listdir(full_path)[random.randint(randint_min, img_count-1)]
        image_name_to_copy, image_extension = os.path.splitext(image_to_copy)
        
        img_original = Image.open(full_path+'/'+image_to_copy)
        width, height = img_original.size
        
        colors = img_original.getpixel((1, 1)) #((random.randint(0, width-1), random.randint(0, height-1)))
        r = colors[0]
        g = colors[1]
        b = colors[2]
        

        create_image(
            full_path, 
            image_name_to_copy+'_'+str(i), 
            r = r, 
            g = g, 
            b = b, 
            brightness=random.randint(9, 14)/10)        
        
        
        print(c, image_to_copy, image_name_to_copy, image_extension, r, g, b)
        
        if images_count(full_path) >= max_images_count:
            break


blue 200
./dataset/images/blue Aqua_1_3_200 0 255 255 1.0
blue Aqua_1_3.png Aqua_1_3 .png 0 255 255
./dataset/images/blue Blue_1_0_low_201 0 0 255 0.9
blue Blue_1_0_low.png Blue_1_0_low .png 0 0 255
./dataset/images/blue RoyalBlue_1_2_202 65 105 225 1.0
blue RoyalBlue_1_2.png RoyalBlue_1_2 .png 65 105 225
./dataset/images/blue LightSkyBlue_1_2_203 135 206 250 1.2
blue LightSkyBlue_1_2.png LightSkyBlue_1_2 .png 135 206 250
./dataset/images/blue MediumSlateBlue_1_0_204 123 104 238 1.1
blue MediumSlateBlue_1_0.png MediumSlateBlue_1_0 .png 123 104 238
./dataset/images/blue SkyBlue_0_9_205 135 206 235 1.3
blue SkyBlue_0_9.png SkyBlue_0_9 .png 135 206 235
./dataset/images/blue RoyalBlue_1_2_202_1_0_206 65 105 225 1.3
blue RoyalBlue_1_2_202_1_0.png RoyalBlue_1_2_202_1_0 .png 65 105 225
./dataset/images/blue DarkBlue_1_2_low_207 0 0 139 1.2
blue DarkBlue_1_2_low.png DarkBlue_1_2_low .png 0 0 139
./dataset/images/blue DarkTurquoise_1_0_low_208 0 206 209 1.4
blue DarkTurquoise_1_0_low.png DarkTu

./dataset/images/blue SteelBlue_1_2_low_220_1_4_low_267_1_4_low_273 70 130 180 1.2
blue SteelBlue_1_2_low_220_1_4_low_267_1_4_low.png SteelBlue_1_2_low_220_1_4_low_267_1_4_low .png 70 130 180
./dataset/images/blue DarkTurquoise_1_0_258_1_2_low_274 0 206 209 1.1
blue DarkTurquoise_1_0_258_1_2_low.png DarkTurquoise_1_0_258_1_2_low .png 0 206 209
./dataset/images/blue DarkTurquoise_1_2_234_0_9_low_275 0 206 209 1.2
blue DarkTurquoise_1_2_234_0_9_low.png DarkTurquoise_1_2_234_0_9_low .png 0 206 209
./dataset/images/blue Aqua_1_3_276 0 255 255 1.1
blue Aqua_1_3.png Aqua_1_3 .png 0 255 255
./dataset/images/blue DarkBlue_0_9_low_277 0 0 139 1.1
blue DarkBlue_0_9_low.png DarkBlue_0_9_low .png 0 0 139
./dataset/images/blue Cyan_1_0_low_278 0 255 255 1.1
blue Cyan_1_0_low.png Cyan_1_0_low .png 0 255 255
./dataset/images/blue RoyalBlue_1_0_279 65 105 225 1.1
blue RoyalBlue_1_0.png RoyalBlue_1_0 .png 65 105 225
./dataset/images/blue DeepSkyBlue_1_3_low_280 0 191 255 1.2
blue DeepSkyBlue_1_3_low.pn

./dataset/images/blue DarkBlue_0_9_low_277_1_1_347 0 0 139 1.2
blue DarkBlue_0_9_low_277_1_1.png DarkBlue_0_9_low_277_1_1 .png 0 0 139
./dataset/images/blue DarkTurquoise_1_2_234_0_9_348 0 206 209 1.2
blue DarkTurquoise_1_2_234_0_9.png DarkTurquoise_1_2_234_0_9 .png 0 206 209
./dataset/images/blue Blue_1_0_low_201_0_9_low_312_1_2_349 0 0 255 1.2
blue Blue_1_0_low_201_0_9_low_312_1_2.png Blue_1_0_low_201_0_9_low_312_1_2 .png 0 0 255
./dataset/images/blue DarkBlue_1_1_low_262_1_3_350 0 0 139 1.1
blue DarkBlue_1_1_low_262_1_3.png DarkBlue_1_1_low_262_1_3 .png 0 0 139
./dataset/images/blue CornflowerBlue_1_1_low_325_1_0_345_0_9_low_351 100 149 237 1.2
blue CornflowerBlue_1_1_low_325_1_0_345_0_9_low.png CornflowerBlue_1_1_low_325_1_0_345_0_9_low .png 100 149 237
./dataset/images/blue SteelBlue_1_2_low_220_1_4_low_267_1_4_352 70 130 180 1.1
blue SteelBlue_1_2_low_220_1_4_low_267_1_4.png SteelBlue_1_2_low_220_1_4_low_267_1_4 .png 70 130 180
./dataset/images/blue Blue_1_0_low_353 0 0 255 1.1
b

blue DarkBlue_0_9_low_277_1_1_311_1_4_365_1_0_low.png DarkBlue_0_9_low_277_1_1_311_1_4_365_1_0_low .png 0 0 139
./dataset/images/blue RoyalBlue_1_2_202_1_0_low_404 65 105 225 1.4
blue RoyalBlue_1_2_202_1_0_low.png RoyalBlue_1_2_202_1_0_low .png 65 105 225
./dataset/images/blue Cyan_1_1_405 0 255 255 0.9
blue Cyan_1_1.png Cyan_1_1 .png 0 255 255
./dataset/images/blue Cyan_1_1_low_231_1_2_low_266_0_9_297_1_4_low_406 0 255 255 1.1
blue Cyan_1_1_low_231_1_2_low_266_0_9_297_1_4_low.png Cyan_1_1_low_231_1_2_low_266_0_9_297_1_4_low .png 0 255 255
./dataset/images/blue Blue_1_0_low_407 0 0 255 1.1
blue Blue_1_0_low.png Blue_1_0_low .png 0 0 255
./dataset/images/blue DarkBlue_0_9_364_0_9_408 0 0 139 0.9
blue DarkBlue_0_9_364_0_9.png DarkBlue_0_9_364_0_9 .png 0 0 139
./dataset/images/blue Aqua_1_3_200_1_0_low_209_1_1_low_223_1_3_low_373_1_2_409 0 255 255 1.4
blue Aqua_1_3_200_1_0_low_209_1_1_low_223_1_3_low_373_1_2.png Aqua_1_3_200_1_0_low_209_1_1_low_223_1_3_low_373_1_2 .png 0 255 255
./dataset

./dataset/images/blue Blue_1_0_low_353_1_1_354_1_0_457 0 0 255 1.1
blue Blue_1_0_low_353_1_1_354_1_0.png Blue_1_0_low_353_1_1_354_1_0 .png 0 0 255
./dataset/images/blue MidnightBlue_1_2_224_1_4_low_237_0_9_low_458 25 25 112 1.1
blue MidnightBlue_1_2_224_1_4_low_237_0_9_low.png MidnightBlue_1_2_224_1_4_low_237_0_9_low .png 25 25 112
./dataset/images/blue MediumSlateBlue_1_0_204_1_1_low_217_1_2_low_222_1_1_low_459 123 104 238 1.1
blue MediumSlateBlue_1_0_204_1_1_low_217_1_2_low_222_1_1_low.png MediumSlateBlue_1_0_204_1_1_low_217_1_2_low_222_1_1_low .png 123 104 238
./dataset/images/blue Aqua_0_9_238_0_9_low_460 0 255 255 1.0
blue Aqua_0_9_238_0_9_low.png Aqua_0_9_238_0_9_low .png 0 255 255
./dataset/images/blue Blue_1_0_low_201_0_9_low_461 0 0 255 1.3
blue Blue_1_0_low_201_0_9_low.png Blue_1_0_low_201_0_9_low .png 0 0 255
./dataset/images/blue CornflowerBlue_1_1_low_325_1_0_443_1_4_462 100 149 237 1.0
blue CornflowerBlue_1_1_low_325_1_0_443_1_4.png CornflowerBlue_1_1_low_325_1_0_443_1_4 

...

yellow Gold_0_9_113_1_2.png Gold_0_9_113_1_2 .png 255 215 0
./dataset/images/yellow Yellow_1_2_low_61_1_3_low_200_1_2_234_1_3_low_402_1_1_low_882 255 255 0 1.4
yellow Yellow_1_2_low_61_1_3_low_200_1_2_234_1_3_low_402_1_1_low.png Yellow_1_2_low_61_1_3_low_200_1_2_234_1_3_low_402_1_1_low .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_767_0_9_797_1_3_883 255 255 0 1.2
yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_767_0_9_797_1_3.png Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_767_0_9_797_1_3 .png 255 255 0
./dataset/images/yellow Yellow_1_2_low_61_1_3_low_72_1_3_231_0_9_282_1_1_low_884 255 255 0 1.3
yellow Yellow_1_2_low_61_1_3_low_72_1_3_231_0_9_282_1_1_low.png Yellow_1_2_low_61_1_3_low_72_1_3_231_0_9_282_1_1_low .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_767_0_9_885 255 255 0 1.1
yellow Gold_0_9_113_1_2_195_1

yellow PaleGoldenrod_1_3_low_80_1_4_165_1_0_232_1_2_312_1_3.png PaleGoldenrod_1_3_low_80_1_4_165_1_0_232_1_2_312_1_3 .png 238 232 170
./dataset/images/yellow Yellow_1_0_118_1_2_low_154_1_4_low_524_1_3_low_916 255 255 0 1.4
yellow Yellow_1_0_118_1_2_low_154_1_4_low_524_1_3_low.png Yellow_1_0_118_1_2_low_154_1_4_low_524_1_3_low .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_917 255 255 0 0.9
yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9.png Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9 .png 255 255 0
./dataset/images/yellow LightYellow_1_3_low_108_1_4_low_621_1_1_918 255 255 224 1.1
yellow LightYellow_1_3_low_108_1_4_low_621_1_1.png LightYellow_1_3_low_108_1_4_low_621_1_1 .png 255 255 224
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_low_761_1_4_845_1_4_856_0

yellow PaleGoldenrod_0_9.png PaleGoldenrod_0_9 .png 238 232 170
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_785_1_2_low_847_0_9_low_949 255 255 0 1.2
yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_785_1_2_low_847_0_9_low.png Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_785_1_2_low_847_0_9_low .png 255 255 0
./dataset/images/yellow PaleGoldenrod_0_9_low_116_1_1_low_136_1_1_low_210_1_2_873_0_9_950 238 232 170 0.9
yellow PaleGoldenrod_0_9_low_116_1_1_low_136_1_1_low_210_1_2_873_0_9.png PaleGoldenrod_0_9_low_116_1_1_low_136_1_1_low_210_1_2_873_0_9 .png 238 232 170
./dataset/images/yellow PaleGoldenrod_0_9_low_116_1_1_low_136_1_1_low_210_1_2_417_1_1_558_1_0_568_1_0_low_610_1_2_951 238 232 170 0.9
yellow PaleGoldenrod_0_9_low_116_1_1_low_136_1_1_low_210_1_2_417_1_1_558_1_0_568_1_0_low_610_1_2.png PaleGoldenrod_0_9_low_116_1_1_low_136_1_1_low_210_1_2_417_1_1_558_1_0_568_1_0_low_610_1_2 .png 238 232 170
.

yellow PaleGoldenrod_1_3_low_96_1_4_120_1_0.png PaleGoldenrod_1_3_low_96_1_4_120_1_0 .png 238 232 170
./dataset/images/yellow Yellow_1_2_low_61_1_3_low_72_1_3_231_0_9_low_560_1_2_926_0_9_low_982 255 255 0 0.9
yellow Yellow_1_2_low_61_1_3_low_72_1_3_231_0_9_low_560_1_2_926_0_9_low.png Yellow_1_2_low_61_1_3_low_72_1_3_231_0_9_low_560_1_2_926_0_9_low .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_983 255 215 0 1.3
yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3.png Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3 .png 255 215 0
./dataset/images/yellow Yellow_1_2_low_61_1_3_112_0_9_low_339_1_3_404_1_2_low_438_0_9_530_1_2_984 255 255 0 1.4
yellow Yellow_1_2_low_61_1_3_112_0_9_low_339_1_3_404_1_2_low_438_0_9_530_1_2.png Yellow_1_2_low_61_1_3_112_0_9_low_339_1_3_404_1_2_low_438_0_9_530_1_2 .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_low_761_1_4_845_1_4_875_1

yellow Yellow_1_0_75_1_2_106_1_2_low_688_0_9_952_1_3_low.png Yellow_1_0_75_1_2_106_1_2_low_688_0_9_952_1_3_low .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_748_1_3_low_989_1_3_1013 255 255 0 0.9
yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_748_1_3_low_989_1_3.png Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_748_1_3_low_989_1_3 .png 255 255 0
./dataset/images/yellow Yellow_1_0_75_1_2_106_1_2_266_1_4_1014 255 255 0 1.4
yellow Yellow_1_0_75_1_2_106_1_2_266_1_4.png Yellow_1_0_75_1_2_106_1_2_266_1_4 .png 255 255 0
./dataset/images/yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_748_1_3_low_989_1_3_1015 255 255 0 1.2
yellow Gold_0_9_113_1_2_195_1_0_271_1_0_302_1_1_397_1_3_505_1_4_569_1_3_low_649_0_9_659_0_9_667_1_1_748_1_3_low_989_1_3.png Gold_0_9_113_1_